In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5792, done.
remote: Counting objects: 100% (981/981), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 5792 (delta 833), reused 811 (delta 695), pack-reused 4811
Receiving objects: 100% (5792/5792), 2.94 GiB | 24.78 MiB/s, done.
Resolving deltas: 100% (4659/4659), done.
Updating files: 100% (492/492), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.9 MB/s eta 0:00:00


In [3]:
# ! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hhp.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hhp.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

Original Paper for HHP: https://arxiv.org/pdf/0808.2670.pdf

https://arxiv.org/pdf/2004.10393.pdf

In [14]:
import numpy as np
import scipy.sparse as sps

from sklearn.preprocessing import normalize
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit

from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Incremental_Similarity_Builder
import time, sys


class HHPRecommender(BaseItemSimilarityMatrixRecommender):
    '''
    HHP (Hybrid HeatS - ProbS) is an hybrid of ProbS (also called P3) and HeatS that uses lambda
    to weight the two contributions. While ProbS implements a random walk with 3 steps, HeatS uses the
    out degree of the neighboring objects.
    
    Original Paper: https://arxiv.org/pdf/0808.2670.pdf
    '''
    
    RECOMMENDER_NAME = "HHPRecommender"
    
    def __init__(self, URM_train):
        super(HHPRecommender, self).__init__(URM_train)
        
    def fit(self, hybrid_lambda=0.5, topK=100, min_rating=0, implicit=False, normalize_similarity=True):
        
        self.hybrid_lambda = hybrid_lambda
        self.topK = topK
        self.min_rating = min_rating
        self.implicit = implicit
        self.normalize_similarity = normalize_similarity

        if self.min_rating > 0:
            self.URM_train.data[self.URM_train.data < self.min_rating] = 0
            self.URM_train.eliminate_zeros()
            if self.implicit:
                self.URM_train.data = np.ones(self.URM_train.data.size, dtype=np.float32)
                
        k_u = np.ediff1d(sps.csr_matrix(self.URM_train).indptr) # user out degree
        k_i = np.ediff1d(sps.csc_matrix(self.URM_train).indptr) # item out degree
        k_i_1_mius_lamda = np.float_power(k_i, 1 - self.hybrid_lambda) + 1e-6
        
        Pui = normalize(self.URM_train, norm='l1', axis=1)
        
        X_bool = self.URM_train.transpose(copy=True)
        X_bool.data = np.ones(X_bool.data.size, np.float32)
        Piu_lambda = X_bool.multiply((1 / (np.float_power(k_i, self.hybrid_lambda) + 1e-6)).reshape(-1, 1))
        Piu_lambda = Piu_lambda.tocsr()

        # Final matrix is computed as Pui * Piu * Pui
        # Multiplication unpacked for memory usage reasons
        block_dim = 200
        d_t = Piu_lambda

        similarity_builder = Incremental_Similarity_Builder(Pui.shape[1], initial_data_block=Pui.shape[1]*self.topK, dtype = np.float32)

        start_time = time.time()
        start_time_printBatch = start_time

        for current_block_start_row in range(0, Pui.shape[1], block_dim):

            if current_block_start_row + block_dim > Pui.shape[1]:
                block_dim = Pui.shape[1] - current_block_start_row

            similarity_block = d_t[current_block_start_row:current_block_start_row + block_dim, :] * Pui
            heatS_contribution = 1 / k_i_1_mius_lamda[current_block_start_row:current_block_start_row + block_dim]
            similarity_block = similarity_block.multiply(heatS_contribution.reshape(-1, 1)).toarray()

            for row_in_block in range(block_dim):
                row_data = similarity_block[row_in_block, :]
                row_data[current_block_start_row + row_in_block] = 0

                relevant_items_partition = np.argpartition(-row_data, self.topK-1, axis=0)[:self.topK]
                row_data = row_data[relevant_items_partition]

                # Incrementally build sparse matrix, do not add zeros
                if np.any(row_data == 0.0):
                    non_zero_mask = row_data != 0.0
                    relevant_items_partition = relevant_items_partition[non_zero_mask]
                    row_data = row_data[non_zero_mask]

                similarity_builder.add_data_lists(row_list_to_add=np.ones(len(row_data), dtype = np.int) * (current_block_start_row + row_in_block),
                                                  col_list_to_add=relevant_items_partition,
                                                  data_list_to_add=row_data)


            if time.time() - start_time_printBatch > 300 or current_block_start_row + block_dim == Pui.shape[1]:
                new_time_value, new_time_unit = seconds_to_biggest_unit(time.time() - start_time)

                self._print("Similarity column {} ({:4.1f}%), {:.2f} column/sec. Elapsed time {:.2f} {}".format(
                     current_block_start_row + block_dim,
                    100.0 * float( current_block_start_row + block_dim) / Pui.shape[1],
                    float( current_block_start_row + block_dim) / (time.time() - start_time),
                    new_time_value, new_time_unit))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        self.W_sparse = similarity_builder.get_SparseMatrix()

        if self.normalize_similarity:
            self.W_sparse = normalize(self.W_sparse, norm='l1', axis=1)

        if self.topK != False:
            self.W_sparse = similarityMatrixTopK(self.W_sparse, k=self.topK)

        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [15]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10]) 

recommender = HHPRecommender(URM_train)
recommender.fit()
result_df, _ = evaluator.evaluateRecommender(recommender)
result_df

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
HHPRecommender: URM Detected 627 ( 4.8%) users with no interactions.
HHPRecommender: URM Detected 247 ( 1.1%) items with no interactions.
HHPRecommender: Similarity column 22347 (100.0%), 1615.96 column/sec. Elapsed time 13.83 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 7.67 sec. Users per second: 1363


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08091,0.138453,0.116416,0.041412,0.071301,0.2436,0.129447,0.095469,0.468057,0.31377,...,0.802826,0.375768,0.802826,0.079619,10.706951,0.998636,0.229732,0.824822,1.402617,0.309225


In [16]:
def objective_function_hhp(trial):

    recommender_instance = HHPRecommender(URM_train)
    recommender_instance.fit(
        hybrid_lambda = trial.suggest_float('hybrid_lambda', 0.01, 0.99),
        topK = trial.suggest_int('topK', 5, 2000, log=True)
    ) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

In [17]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='tuning',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_hhp, n_trials=200)

[I 2023-12-24 15:36:22,114] A new study created in RDB with name: tuning


HHPRecommender: URM Detected 627 ( 4.8%) users with no interactions.
HHPRecommender: URM Detected 247 ( 1.1%) items with no interactions.
HHPRecommender: Similarity column 22347 (100.0%), 1037.56 column/sec. Elapsed time 21.54 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 9.28 sec. Users per second: 1127


[I 2023-12-24 15:37:00,070] Trial 0 finished with value: 0.03927076501985689 and parameters: {'hybrid_lambda': 0.6657383708030581, 'topK': 420}. Best is trial 0 with value: 0.03927076501985689.


HHPRecommender: URM Detected 627 ( 4.8%) users with no interactions.
HHPRecommender: URM Detected 247 ( 1.1%) items with no interactions.
HHPRecommender: Similarity column 22347 (100.0%), 2280.28 column/sec. Elapsed time 9.80 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 7.73 sec. Users per second: 1352


[I 2023-12-24 15:37:19,786] Trial 1 finished with value: 0.04123409814065959 and parameters: {'hybrid_lambda': 0.05683460467977674, 'topK': 118}. Best is trial 1 with value: 0.04123409814065959.


...

[I 2023-12-24 16:46:07,618] Trial 197 finished with value: 0.04137597005137231 and parameters: {'hybrid_lambda': 0.06507436816538156, 'topK': 102}. Best is trial 192 with value: 0.04141740597029482.


HHPRecommender: URM Detected 627 ( 4.8%) users with no interactions.
HHPRecommender: URM Detected 247 ( 1.1%) items with no interactions.
HHPRecommender: Similarity column 22347 (100.0%), 2001.30 column/sec. Elapsed time 11.17 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 8.15 sec. Users per second: 1282


[I 2023-12-24 16:46:30,082] Trial 198 finished with value: 0.041217099470495136 and parameters: {'hybrid_lambda': 0.09886049369635685, 'topK': 170}. Best is trial 192 with value: 0.04141740597029482.


HHPRecommender: URM Detected 627 ( 4.8%) users with no interactions.
HHPRecommender: URM Detected 247 ( 1.1%) items with no interactions.
HHPRecommender: Similarity column 22347 (100.0%), 2117.37 column/sec. Elapsed time 10.55 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 8.08 sec. Users per second: 1295


[I 2023-12-24 16:46:51,282] Trial 199 finished with value: 0.04135495606684448 and parameters: {'hybrid_lambda': 0.14651451733870155, 'topK': 142}. Best is trial 192 with value: 0.04141740597029482.


In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [20]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [21]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hhp.json', 'w') as params_file:
        params = study.best_params
        json.dump(params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hhp.json', 
            'tuning_results/best_params_hhp.json', 
            'HHP PyTorch tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/best_params_hhp.json',
        '/kaggle/working/best_params_hhp.json'
    )

In [22]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_hhp.db',
        'HHP tuning db updated results (from kaggle notebook)'
    )